# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "../output_data/weather_data_df.csv"
# output_data_file

types_df = pd.read_csv(output_data_file)

weather_data_df = types_df.drop(columns=["Unnamed: 0"])
weather_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Dingle,86,PH,1586981271,75,11.00,122.67,77.40,5.64
1,Port Hedland,98,AU,1586981272,88,-20.32,118.57,78.80,5.82
2,Saskylakh,98,RU,1586981273,90,71.92,114.08,-6.32,5.82
3,Chu,93,KZ,1586981274,50,43.60,73.76,44.31,6.46
4,Avarua,92,CK,1586981275,69,-21.21,-159.78,80.60,4.70
...,...,...,...,...,...,...,...,...,...
549,Bijapur,42,IN,1586981900,50,16.83,75.70,79.02,5.30
550,Ajka,0,HU,1586981901,56,47.10,17.56,48.20,4.70
551,Mandaon,98,PH,1586981902,78,12.23,123.28,81.70,8.23
552,Russell,1,US,1586981904,29,32.35,-85.20,68.00,12.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_data_df[["Lat", "Lng"]]

humidity = weather_data_df["Humidity"].astype(float)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weather_new_df = pd.DataFrame(weather_data_df)

max_temp = (weather_new_df["Max Temp"] <= 80) & (weather_new_df["Max Temp"] > 70)
wind_speed = weather_new_df["Wind Speed"] < 10
cloudiness = weather_new_df["Cloudiness"] == 0


final_new_df = weather_new_df[max_temp & max_temp & cloudiness]
final_new_df.dropna()
final_new_df.groupby('City')
final_new_df.reset_index(drop=True, inplace=True)
final_new_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Rio do Sul,0,BR,1586981102,36,-27.21,-49.64,77.00,1.01
1,Atar,0,MR,1586980847,16,20.52,-13.05,78.19,6.55
2,Pisco,0,PE,1586981384,30,-13.70,-76.22,75.99,13.87
3,Opuwo,0,NaN,1586981414,32,-18.06,13.84,70.48,3.67
4,Birao,0,CF,1586981445,13,10.28,22.79,79.39,7.07
5,Umluj,0,SA,1586981487,50,25.02,37.27,74.23,8.55
6,Terenos,0,BR,1586981603,33,-20.44,-54.86,73.40,8.05
7,Saint-Pierre,0,RE,1586981612,69,-21.34,55.48,75.20,6.93
8,Kovūr,0,IN,1586981647,88,14.48,79.98,74.95,8.97
9,Najrān,0,SA,1586981384,53,17.49,44.13,75.20,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(final_new_df)

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for City {index+1}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for City 1: Rio do Sul.
Closest hotel is Hotel | Restaurante Aliança Express.
------------
Retrieving Results for City 2: Atar.
Closest hotel is Oumou Elghoura.
------------
Retrieving Results for City 3: Pisco.
Closest hotel is Hostal San Isidro Oficial.
------------
Retrieving Results for City 4: Opuwo.
Closest hotel is ABBA Guesthouse.
------------
Retrieving Results for City 5: Birao.
Missing field/result... skipping.
------------
Retrieving Results for City 6: Umluj.
Closest hotel is Tanal Hotel Apartment.
------------
Retrieving Results for City 7: Terenos.
Closest hotel is QUINTAS IN CASA.
------------
Retrieving Results for City 8: Saint-Pierre.
Closest hotel is Lindsey Hôtel.
------------
Retrieving Results for City 9: Kovūr.
Closest hotel is Pleasant Stay.
------------
Retrieving Results for City 10: Najrān.
Closest hotel is محمد هادي ال فطيح.
------------
Retrieving Results for City 11: Hwange.
Closest hotel is Hwange Colliery Guest House.
------------
Ret

In [7]:
hotel_df.dropna()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Rio do Sul,0,BR,1586981102,36,-27.21,-49.64,77.00,1.01,Hotel | Restaurante Aliança Express
1,Atar,0,MR,1586980847,16,20.52,-13.05,78.19,6.55,Oumou Elghoura
2,Pisco,0,PE,1586981384,30,-13.70,-76.22,75.99,13.87,Hostal San Isidro Oficial
5,Umluj,0,SA,1586981487,50,25.02,37.27,74.23,8.55,Tanal Hotel Apartment
6,Terenos,0,BR,1586981603,33,-20.44,-54.86,73.40,8.05,QUINTAS IN CASA
7,Saint-Pierre,0,RE,1586981612,69,-21.34,55.48,75.20,6.93,Lindsey Hôtel
8,Kovūr,0,IN,1586981647,88,14.48,79.98,74.95,8.97,Pleasant Stay
9,Najrān,0,SA,1586981384,53,17.49,44.13,75.20,6.93,محمد هادي ال فطيح
10,Hwange,0,ZW,1586981680,44,-18.37,26.50,73.04,0.76,Hwange Colliery Guest House
11,Saint-Leu,0,RE,1586981807,73,-21.15,55.28,75.20,6.93,La Caz Oceane


In [8]:

# Using the template to add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

locations = hotel_df[["Lat", "Lng"]]

markers = gmaps.marker_layer(locations)

mapping_hotel = gmaps.symbol_layer(locations, info_box_content = hotel_info, scale = 2)

In [9]:
# Add marker layer ontop of heat map

fig.add_layer(markers)
fig.add_layer(mapping_hotel)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))